<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/main/Bag_Of_Words_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def clean_text(sentences):
  cleaned_sentence = ""
  sentences = sentences.lower().split()
  ##removing stop words
  words = [i for i in sentences if i not in stopwords.words('english')]
  words = " ".join(words)               ##joining our words back to sentences
  return words

In [3]:
def classify(length, ngram, count, label):
  X = df['selftext'][0:length].tolist()
  Y = df['labels'][0:length].tolist()
  for i, post in enumerate(X):
    X[i] = clean_text(post)
  if ngram == "binary":
    vectorizer = CountVectorizer(binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer()
  else:
    vectorizer = CountVectorizer(ngram_range = (2, 2))
  bag = vectorizer.fit_transform(X)
  bag = bag.toarray()

  skf = StratifiedKFold(n_splits=10)
  skf.get_n_splits(bag, Y)

  for i, (train_index, test_index) in enumerate(skf.split(bag, Y)):
    # print(i)
    X_train, y_train, X_test, y_test = [], [], [], []
    for i in train_index:
      X_train.append(bag[i])
      y_train.append(Y[i])
    for i in test_index:
      X_test.append(bag[i])
      y_test.append(Y[i])
    (X_train, y_train, X_test, y_test) = map(np.array, (X_train, y_train, X_test, y_test))
  w = {0:2000-count, 1:count}
  lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter = 100000, class_weight = w)
  #
  # Fit the model
  #
  lr.fit(X_train, y_train)
  #
  # Create the predictions
  #
  y_predict = lr.predict(X_test)

  # Use metrics.accuracy_score to measure the score
  #'macro' favors minority, 'weighted' favors majority, 'micro' favors none (when you have multiclass)
  if label in [1.0, 4.0]:
    score = f1_score(y_test, y_predict, average='micro', labels=np.unique(y_predict))
  else:
    score = f1_score(y_test, y_predict, average='macro', labels=np.unique(y_predict))
  return "%.3f" %score

In [ ]:
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
# for i in list(set(df.labels))[:6]:
for i in [1.0, 2.0, 3.0, 4.0]
  count = 0
  for ind in df.index:
    # float 0.0, 1.0, 2.0, 3.0, 4.0, 5.0 - represents: general, question, advice, encouragement, experience, bragging posts
    if df.loc[ind, "labels"] != i:
      df.loc[ind, "labels"] = 0
    else:
      df.loc[ind, "labels"] = 1
      count += 1
  length = 100
  unigram, bigram, binary = [], [], []
  while length < 2100:
    unigram.append(classify(length, "unigram", count, i))
    bigram.append(classify(length, "bigram", count, i))
    binary.append(classify(length, "binary", count, i))
    length += 100
  print("Scores for", i, ": \n")
  for i in unigram:
    print(i)
  print("-----------------------")
  for i in bigram:
    print(i)
  print("-----------------------")
  for i in binary:
    print(i)
  print("-----------------------")
  print("\n")
  df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')

Scores for 1.0 : 

0.700
0.650
0.500
0.700
0.800
0.750
0.686
0.750
0.633
0.700
0.682
0.775
0.723
0.686
0.747
0.706
0.747
0.761
0.711
0.665
-----------------------
0.600
0.400
0.533
0.600
0.680
0.717
0.614
0.662
0.633
0.620
0.645
0.683
0.677
0.700
0.640
0.675
0.665
0.694
0.679
0.615
-----------------------
0.600
0.650
0.567
0.650
0.860
0.733
0.643
0.700
0.644
0.680
0.700
0.775
0.731
0.686
0.713
0.688
0.718
0.756
0.763
0.710
-----------------------




In [ ]:
#test to see how many of each post there are
import pandas as pd
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
labels = list(df.labels)
questions = labels.count(1.0)
advice = labels.count(2.0)
encouragement  = labels.count(3.0)
experience = labels.count(4.0)
bragging = labels.count(5.0)
print(questions, advice, encouragement, experience, bragging)
print(labels.count(0.0))
ratio = {0:2000-experience, 1:experience}
print(ratio)

1046 103 90 675 25
61
{0: 1325, 1: 675}
